In [1]:
!pip install torch lightning numpy kaggle wandb torch-geometric polars


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.6 MB/s eta 0:00:00


In [79]:
!pip install polars -U
!pip install torch-geometric -U

In [3]:
from google.colab import files

# Carica il file kaggle.json
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"alexxxyy47","key":"29be390563ab164c9d7671a10a031097"}'}

In [4]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [5]:
!kaggle datasets download -d ealaxi/paysim1
!unzip paysim1.zip
!rm paysim1.zip

100% 178M/178M [00:07<00:00, 29.8MB/s]
100% 178M/178M [00:07<00:00, 25.7MB/s]
Archive:  paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [1]:
import pandas as pd, sys, plotly.graph_objects as go, plotly.express as px, numpy as np, torch, random as rnd, torch.nn as nn, lightning as L, wandb as wndb, pdb
from torch.utils.data import Dataset, DataLoader
from sklearn.utils import shuffle
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
from matplotlib import pyplot as plt
import networkx as nx
from torch_geometric.nn import GCNConv
import torch_geometric.transforms as T
from torch import optim
from torchmetrics.classification import BinaryF1Score
from torch_geometric.nn import MessagePassing
from typing import Optional
from torch.nn import Parameter
from torch_geometric.typing import (
    Adj,
    OptPairTensor,
    OptTensor,
    SparseTensor,
    torch_sparse,
)
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.utils import add_self_loops as add_self_loops_fn
from torch_geometric.utils import (
    is_torch_sparse_tensor,
    scatter,
    spmm,
    to_edge_index,
)
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_geometric.utils.sparse import set_sparse_value
from torch_geometric.nn.dense.linear import Linear
from torch_geometric.nn.inits import zeros
import polars as pl, json
from torch_geometric.nn import Sequential
from torch_geometric import seed_everything




In [2]:
# PARAMETERS

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
ACCELERATOR =  "gpu" if torch.cuda.is_available() else "cpu"
SEED = 42

rnd.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
seed_everything(SEED)

DEPTH = 2
LR = 1e-3
WD = 1e-5
H_SIZE = 16

"aaf831dabc88d936d4e6b439b798bb4cb42814ea"

#wndb.login()



'aaf831dabc88d936d4e6b439b798bb4cb42814ea'

In [3]:
# UTILS FUNCTIONS

def load_dataframe( dataset_file : str):
    return pl.read_csv(dataset_file)


def find_null_or_empty_records( dataframe):
    return # dataframe.filter(dataframe.apply(lambda col: col.is_nan().any()))

def print_progress_bar(percentuale, lunghezza_barra=20):
    blocchi_compilati = int(lunghezza_barra * percentuale)
    barra = "[" + "=" * (blocchi_compilati - 1) + ">" + " " * (lunghezza_barra - blocchi_compilati) + "]"
    sys.stdout.write(f"\r{barra} {percentuale * 100:.2f}% completo")
    sys.stdout.flush()


def compute_kind_inconsistence(dataframe):
    return {"inconsistent orig balance": len(dataframe.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')))/len(dataframe),
            "inconsistent dest balance": len(dataframe.filter(abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')))/len(dataframe),
            "zero cash transaction": len(dataframe.filter(pl.col('amount') == 0 ))/len(dataframe),
            "self-transaction": len(dataframe.filter(pl.col('nameOrig') == pl.col('nameDest')))/len(dataframe)
            }

def plot_histogram(to_plot):


    # Converti il dizionario in un array di valori
    values = list(to_plot.values())

    # Crea un istogramma
    fig = go.Figure(data=[go.Bar(x=list(to_plot.keys()), y=values)])

    # Mostra l'istogramma
    fig.show()



def plot_categories(dataframe):
    # Calcola la frequenza di ogni categoria nella colonna 'type'
    counts = dataframe.groupby('type').count()

    # Crea l'istogramma con Plotly Express
    fig = px.bar(counts, x='type', y='count', title='Istogramma delle categorie nella colonna "type"')

    # Mostra il plot
    fig.show()


def build_names_conversion_index(df):
  all_names = pl.DataFrame({"names": pl.concat([df['nameDest'] , df['nameOrig']])}).select("names").unique()
  # Costruisci il dizionario di mapping
  all_names = pl.DataFrame({"names": all_names["names"], "id": [i for i in range(len(all_names["names"]))]})

  try:
    with open("name_to_id.json","r") as f:
      return json.load(f), all_names
  except:
    name_to_id = dict(zip(all_names['names'], all_names['id']))
    with open("name_to_id.json","w") as f:
      json.dump(name_to_id, f)
      #breakpoint()
      return name_to_id , all_names


def compute_mean_var(df, name,var=False):
  try:

    orig = df.filter(pl.col('nameOrig') == name).select(pl.col('newbalanceOrig'))['newbalanceOrig']
    dest = df.filter(pl.col('nameDest') == name).select(pl.col('newbalanceDest'))['newbalanceDest']
    if name == 'C1831599196':
      breakpoint()
    if var:
      return dest.var() if orig.shape[0] == 0 else orig.var() if dest.shape[0] == 0 else (len(orig) * orig.var() + len(dest) * dest.var()) / (len(orig) + len(dest))

    else:
      return dest.mean() if orig.shape[0] == 0 else orig.mean() if dest.shape[0] == 0 else (orig.mean() + dest.mean()) / 2
  except:
    print(name)


def combine_lists(l1,l2):
  l3 = []
  for el in l1:
    if not el is None:
      l3.append(el)
  for el in l2:
    if not el is None:
      l3.append(el)
  return l3

def str_to_list(s):
  l = s.split(',')
  return [float(e) for e in l]


def random_split(df, train_prc, val_prc):

  df_indici = pl.DataFrame({'id': list(range(len(df)))})

  df_ids = pl.concat([df, df_indici], how="horizontal")
  #breakpoint()

  pos = df_ids.filter(pl.col('isFraud') == 1).select(pl.col("id"))
  neg = df_ids.filter(pl.col('isFraud') == 0).select(pl.col("id"))
  val_len_pos = int(len(pos) * val_prc)
  val_len_neg = int(len(neg) * val_prc)
  pos_train_ids = set(rnd.sample(list(pos['id']), int(len(pos) * train_prc)))
  pos_train1 = pos.with_columns(pl.col("id").map_elements(lambda x: x in pos_train_ids ).alias("check"))
  pos_train = pos_train1.filter(pl.col('check') == True)
  pos = pos_train1.filter(pl.col('check') == False).select(pl.col("id"))

  neg_train_ids = set(rnd.sample(list(neg['id']), int( len(neg) * train_prc)))
  neg_train1 = neg.with_columns(pl.col("id").map_elements(lambda x: x in neg_train_ids).alias("check"))
  neg_train = neg_train1.filter(pl.col('check') == True)
  neg = neg_train1.filter(pl.col('check') == False).select(pl.col("id"))

  pos_val_ids = set(rnd.sample(list(pos['id']), val_len_pos))
  pos_val1 = pos.with_columns(pl.col("id").map_elements(lambda x: x in pos_val_ids  ).alias("check"))
  pos_val = pos_val1.filter(pl.col('check') == True)
  pos = pos_val1.filter(pl.col('check') == False).select(pl.col("id"))

  neg_val_ids = set(rnd.sample(list(neg['id']), val_len_neg))
  neg_val1 = neg.with_columns(pl.col("id").map_elements(lambda x: x in neg_val_ids ).alias("check"))
  neg_val = neg_val1.filter(pl.col('check') == True)
  neg = neg_val1.filter(pl.col('check') == False).select(pl.col("id"))

  train_set = pl.concat([pos_train, neg_train])
  val_set = pl.concat([pos_val, neg_val])
  test_set = pl.concat([pos, neg])
  return train_set, val_set, test_set, df_ids













In [4]:
class FraudDetectionDataset(Dataset):

    def __init__(self,dataset_file : str):
        dataframe = load_dataframe(dataset_file)

        dataframe = dataframe.with_columns(pl.col("isFraud").cast(int).alias("isFraud"))
        self.raw_data = dataframe
        self.transaction_types = {
            "CASH_IN": 0,
            "CASH_OUT": 1,
            "DEBIT": 2,
            "PAYMENT": 3,
            "TRANSFER": 4
        }

    def analize_data(self,find_empty_records = True):
        print("----HEAD----")
        print(self.raw_data.head())
        print("----DESCRIBE----")
        print(self.raw_data.describe())
        if find_empty_records:
          find_null_or_empty_records(self.raw_data)

    def extract_inconsistent_transactions(self):

        return self.raw_data.filter(
            (abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')) |
            (abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')) | (pl.col('amount') == 0) | (pl.col('nameOrig') == pl.col('nameDest'))
        )


    def remove_inconsistent(self):
        self.raw_data = self.raw_data.filter( (pl.col('amount') != 0)) #((abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) == abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))) & ~pl.col("nameDest").str.starts_with("M")) &


    def convert_type(self):
      self.raw_data = self.raw_data.select(pl.exclude("isFlaggedFraud"))
      self.raw_data = self.raw_data.with_columns(pl.col("type").map_elements(lambda x: self.transaction_types[x]).alias("type"))



    def build_graph(self, train_prc, val_prc):
      #breakpoint()
      self.train_edges, self.val_edges, self.test_edges, self.raw_data = random_split(self.raw_data, train_prc, val_prc)

      try:
        with open('graph_data.json','r') as f:
          graph = json.load(f)
        edge_attr = torch.tensor(graph['edge_attr'])
        y = torch.tensor(graph['y'])
        edge_index = torch.tensor(graph['edge_index'])
        x = torch.tensor(graph['x'])



      except:
        node_names, x_dataframe = build_names_conversion_index(self.raw_data)
        #breakpoint()/

        join1 = x_dataframe.join(self.raw_data.select(pl.col("nameOrig",'oldbalanceOrg','amount')), left_on='names', right_on='nameOrig',how='left')
        join2 = join1.join(self.raw_data.select(pl.col("nameDest",'oldbalanceDest'),pl.col('amount').alias('amount2')), left_on='names', right_on='nameDest',how='left')
        join3 = join2.with_columns(pl.col("*").fill_null(strategy="zero"))
        del join1
        del join2

        group1 = join3.lazy().group_by("names").agg(pl.col("oldbalanceOrg").alias("grouped_balanceOrig"), pl.col('amount').alias('grouped_amount'),
              pl.col('oldbalanceDest').alias('grouped_balanceDest'), pl.col('amount2').alias('grouped_amount2') )
        del join3
        group2 = group1.collect().with_columns(pl.col('grouped_balanceDest').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_balanceDest'),
                                              pl.col('grouped_balanceOrig').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_balanceOrig'),
                                              pl.col('grouped_amount').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_amount'),
                                              pl.col('grouped_amount2').map_elements(lambda x: str(list(x))[1:-1]).alias('grouped_amount2'))
        breakpoint()
        del group1
        group3 = group2.select(
            pl.col('names'),
            pl.reduce(
              lambda x, y:
              x + "," + y, exprs=pl.col("grouped_balanceOrig","grouped_balanceDest")
          ).alias("balance"),
            pl.reduce(
              lambda x, y:
              x + "," + y, exprs=pl.col("grouped_amount","grouped_amount2")
          ).alias("amount")
          )
        del group2
        group4 = group3.with_columns(
            pl.col('balance').map_elements(lambda x : np.mean(str_to_list(x))).alias('mean_balance'),
            pl.col('balance').map_elements(lambda x : np.var(str_to_list(x))).alias('variance_balance'),
            pl.col('amount').map_elements(lambda x : np.mean(str_to_list(x))).alias('mean_amount'),
            pl.col('amount').map_elements(lambda x : np.var(str_to_list(x))).alias('variance_amount')

        ).select(pl.exclude('amount','balance'))
        del group3





        # x_dataframe.with_columns(
        #                         pl.col("names").map_elements(lambda x: 1 if x.startswith('M') else 0 ).alias("commercial"),
        #                         pl.col("names").map_elements(lambda x: compute_mean_var(self.raw_data, x) ).alias("mean"),
        #                         pl.col("names").map_elements(lambda x: compute_mean_var(self.raw_data, x, var=True) ).alias("var")
        #                         )

        self.raw_data =  self.raw_data.with_columns(pl.col("nameOrig").map_elements(lambda x: node_names[x]).alias("nameOrig"))
        self.raw_data =  self.raw_data.with_columns(pl.col("nameDest").map_elements(lambda x: node_names[x]).alias("nameDest"))
        self.raw_data =  self.raw_data.with_columns(pl.col("step").map_elements(lambda x: x%24).alias("step"))
        x = torch.tensor(group4.select(pl.exclude('names')).to_numpy(), dtype=torch.float)


        #breakpoint()
        chtype1 = self.raw_data.with_columns(pl.col('step').cast(str).alias("step"), pl.col('type').cast(str).alias("type"), pl.col('amount').cast(str).alias("amount"))
        red1 = chtype1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('step','amount') ).alias('edge_attr'), pl.col("type"))
        red2 = red1.select(pl.reduce(lambda x, y: x + y, exprs=pl.col('type','edge_attr') ).alias('edge_attr'))
        chtype2 = red2.with_columns(pl.col('edge_attr').cast(float).alias('edge_attr'))
        edge_attr = torch.tensor(chtype2.to_numpy(), dtype=torch.float)
        y = torch.tensor(self.raw_data.select(pl.col('isFraud')).to_numpy(), dtype=torch.float)

        edge_index = torch.tensor(self.raw_data.select(pl.col('nameOrig', 'nameDest')).to_numpy(), dtype=torch.int).t()
        self.raw_data = None

        json_data = {
            "edge_attr": edge_attr.tolist(),
            "y": y.tolist(),
            "edge_index": edge_index.tolist(),
            "x": x.tolist()
        }

        with open('graph_data.json', 'w') as f:
          json.dump(json_data,f)



      #breakpoint()
      data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
      data.y = y

      #self.split = T.RandomNodeSplit(num_val=val_prc, num_test=test_prc)
      self.data = [data]


    def __getitem__(self, index):
      return self.data[0]

    def __len__(self):
      return 1


    def collate(self, data):

      #breakpoint()
      return data[0],self.train_edges, self.val_edges, self.test_edges



    def get_dataloader(self):
      return DataLoader(self, batch_size=1, shuffle=False, collate_fn = self.collate)












In [5]:
dataset = FraudDetectionDataset("PS_20174392719_1491204439457_log.csv")


In [ ]:
dataset.raw_data.filter((abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount')) | (abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount')))

# dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) != pl.col('amount') |
#             abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest')) != pl.col('amount') | pl.col('amount') == 0 | pl.col('nameOrig').is_in([pl.col('nameDest')])
#         )

In [ ]:
inconsistent_data = dataset.extract_inconsistent_transactions()
d = compute_kind_inconsistence(inconsistent_data)
plot_histogram(d)


In [ ]:
print(dataset.raw_data.columns)
print(len(dataset.raw_data))
dataset.remove_inconsistent()
print(len(dataset.raw_data))
dataset.convert_type()
dataset.build_graph(0.7,0.1)


['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']
6362620
6362604


<ipython-input-3-328f263cf002>:108: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("id").map_elements(lambda x: ...)
with this one instead:
  + pl.col("id").is_in(pos_train_ids)

  pos_train1 = pos.with_columns(pl.col("id").map_elements(lambda x: x in pos_train_ids ).alias("check"))
<ipython-input-3-328f263cf002>:113: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("id").map_elements(lambda x: ...)
with this one instead:
  + pl.col("id").is_in(neg_train_ids)

  neg_train1 = neg.with_columns(pl.col("id").map_elements(lambda x: x in neg_train_ids).alias("check"))
<ipython-input-3-328f263cf002>:118: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower t

In [ ]:
df = pl.DataFrame(
    {
        "a": ["[1]", "[2]", "[3]"],
        "b": ["[0]", "[1]","[2]"],
    }
)

df2 = pl.DataFrame(
    {
        "a": [1, 2, 3],
        "b": [0, 1, 2],
    }
)
def concatenate(acc, x):
  for e in x:
    acc.append(e)
  return acc

df = df.with_columns(
    pl.reduce(function=lambda acc, x: acc +"|"+ x, exprs=pl.col("*")).alias("sum")
)

df2 = df2.with_columns(
    pl.reduce(function=lambda acc, x: acc + x, exprs=pl.col("*")).alias("sum")
)

print(df)
print(df2)


shape: (3, 3)
┌─────┬─────┬─────────┐
│ a   ┆ b   ┆ sum     │
│ --- ┆ --- ┆ ---     │
│ str ┆ str ┆ str     │
╞═════╪═════╪═════════╡
│ [1] ┆ [0] ┆ [1]|[0] │
│ [2] ┆ [1] ┆ [2]|[1] │
│ [3] ┆ [2] ┆ [3]|[2] │
└─────┴─────┴─────────┘
shape: (3, 3)
┌─────┬─────┬─────┐
│ a   ┆ b   ┆ sum │
│ --- ┆ --- ┆ --- │
│ i64 ┆ i64 ┆ i64 │
╞═════╪═════╪═════╡
│ 1   ┆ 0   ┆ 1   │
│ 2   ┆ 1   ┆ 3   │
│ 3   ┆ 2   ┆ 5   │
└─────┴─────┴─────┘


In [ ]:
def convert_to_networkx(graph):
	g = to_networkx(graph, node_attrs=['x'])

	return g

def plot_graph(g):
	plt.figure(figsize=(9,7))
	nx.draw_spring(g, node_size=30, arrows=False)
	plt.show()



g = convert_to_networkx(dataset.data[0])
plot_graph(g)
del g


In [86]:
class FraudDetectionModule(L.LightningModule):

  def __init__(self,nodes_size, hidden_layer_size, depth, lr=1e-3, wd=5e-4):
    super().__init__()

    #self.gnn = Sequential(GCNConv(nodes_size, hidden_layer_size), nn.ReLU())
    layers = [GCNConv(nodes_size, hidden_layer_size)]
    for _ in range(depth-2):
      layers.append(GCNConv(hidden_layer_size, hidden_layer_size))

    self.gnn_act = nn.ReLU()
    self.gnn = layers
    self.classifier = nn.Linear(hidden_layer_size*2+1, 1)
    self.sigmoid = nn.Sigmoid()
    self.lr = lr
    self.wd = wd
    self.validation_step_outputs = []
    self.f1 = BinaryF1Score()

    self.loss = nn.BCEWithLogitsLoss()

  def forward_seq(self,x, edge_index, edge_attr):

    for layer in self.gnn:
      x = layer(x,edge_index, edge_attr)
      x = self.gnn_act(x)
    return x

  def forward(self,data):
    breakpoint()
    edge_index = data[0].edge_index
    edge_attr = data[0].edge_attr
    x = data[0].x
    train_edges = data[1]
    #breakpoint()

    self.forward_seq(x, edge_index, edge_attr)

    train_features = edge_attr[train_edges]
    train_edge_index = edge_index[:,train_edges]
    del edge_attr
    del train_edges
    del edge_index
    #breakpoint()
    #add batch  with edge nodes + features of edges
    from_nodes = torch.nan_to_num(x.squeeze()[train_edge_index[0,:].squeeze()])
    dest_nodes = torch.nan_to_num(x.squeeze()[train_edge_index[1,:].squeeze()])
    del train_edge_index
    to_classify = torch.cat((from_nodes,dest_nodes,train_features), dim=1)


    return self.sigmoid(self.classifier(to_classify))



  def training_step(self, batch, batch_idx):
    #breakpoint()

    #print("training")

    z = self.forward(batch)
    loss = self.loss(z,batch[0].y[batch[1]])
    # Logging to TensorBoard (if installed) by default
    self.log("train_loss", loss, prog_bar=True)
    return loss


  def validation_step(self, batch, batch_idx):
    #breakpoint()

    #print("validation")

    z = self.forward((batch[0], batch[2]))
    print(z.shape)
    val_loss = self.loss(z,batch[0].y[batch[2]])
    f1 = self.f1(z,batch[0].y[batch[2]])
    #wndb.log({"val_loss": val_loss,"f1-score":f1})
    self.log_dict({"val_loss": val_loss,"f1-score":f1}, prog_bar=True)


  def configure_optimizers(self):
    optimizer = optim.Adam(self.parameters(), lr=self.lr, weight_decay=self.wd)
    return optimizer




In [87]:


# wndb.init(
#     project="datamining-hw4",

#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": LR ,
#     "weight decay": WD ,
#     "layers": DEPTH ,
#     "hidden layers size": H_SIZE
#     })
#dataset.to_device()
loader = dataset.get_dataloader()
#breakpoint()
print(dataset.data[0].num_node_features)
model = FraudDetectionModule(dataset.data[0].num_node_features ,H_SIZE,DEPTH,LR,WD)
#model.to(DEVICE)

trainer = L.Trainer(deterministic=True,max_epochs=15,accelerator=ACCELERATOR )



INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


4


In [88]:
# for name, param in model.named_parameters():
    # print(f"Parameter {name} device:", param.device)

trainer.fit(model, loader, loader)
#wndb.finish()

INFO: 
  | Name       | Type              | Params
-------------------------------------------------
0 | gnn_act    | ReLU              | 0     
1 | classifier | Linear            | 34    
2 | sigmoid    | Sigmoid           | 0     
3 | f1         | BinaryF1Score     | 0     
4 | loss       | BCEWithLogitsLoss | 0     
-------------------------------------------------
34        Trainable params
0         Non-trainable params
34        Total params
0.000     Total estimated model params size (MB)
INFO:lightning.pytorch.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | gnn_act    | ReLU              | 0     
1 | classifier | Linear            | 34    
2 | sigmoid    | Sigmoid           | 0     
3 | f1         | BinaryF1Score     | 0     
4 | loss       | BCEWithLogitsLoss | 0     
-------------------------------------------------
34        Trainable params
0         Non-trainable params
34        Total params
0.000 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


> <ipython-input-86-29bac6ed3f0d>(31)forward()
     29   def forward(self,data):
     30     breakpoint()
---> 31     edge_index = data[0].edge_index
     32     edge_attr = data[0].edge_attr
     33     x = data[0].x

ipdb> n
> <ipython-input-86-29bac6ed3f0d>(32)forward()
     30     breakpoint()
     31     edge_index = data[0].edge_index
---> 32     edge_attr = data[0].edge_attr
     33     x = data[0].x
     34     train_edges = data[1]

ipdb> n
> <ipython-input-86-29bac6ed3f0d>(33)forward()
     31     edge_index = data[0].edge_index
     32     edge_attr = data[0].edge_attr
---> 33     x = data[0].x
     34     train_edges = data[1]
     35     #breakpoint()

ipdb> n
> <ipython-input-86-29bac6ed3f0d>(34)forward()
     32     edge_attr = data[0].edge_attr
     33     x = data[0].x
---> 34     train_edges = data[1]
     35     #breakpoint()
     36 

ipdb> n
> <ipython-input-86-29bac6ed3f0d>(37)forward()
     35     #breakpoint()
     36 
---> 37     self.forward_seq(x, edge_index,

In [ ]:
dataset.analize_data(find_empty_records=False)


# Calcola la percentuale di record fraudolenti in inconsistent_data
print("inconsistent fraud")
print(inconsistent_data.filter(pl.col('isFraud') == 1).height / inconsistent_data.height)

# Calcola la percentuale di record non fraudolenti in inconsistent_data
print("inconsistent not fraud")
print(inconsistent_data.filter(pl.col('isFraud') == 0).height / inconsistent_data.height)

# Calcola la percentuale di record con oldbalanceOrg - newbalanceOrig > oldbalanceDest - newbalanceDest
print("given > gotten")
print(dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) > abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))).height / inconsistent_data.height)

# Calcola la percentuale di record con oldbalanceOrg - newbalanceOrig < oldbalanceDest - newbalanceDest
print("given < gotten")
print(dataset.raw_data.filter(abs(pl.col('oldbalanceOrg') - pl.col('newbalanceOrig')) < abs(pl.col('oldbalanceDest') - pl.col('newbalanceDest'))).height / inconsistent_data.height)

# Calcola il numero di valori distinti nella colonna 'nameOrig'
print("distinct names")
print( pl.DataFrame({"names": pl.concat([dataset.raw_data['nameDest'] , dataset.raw_data['nameOrig']])}).select("names").unique().height)


# Calcola la percentuale di record con isFlaggedFraud e isFraud contemporaneamente
print("isFlaggedFraud and isFraud")
print(dataset.raw_data.filter((pl.col('isFlaggedFraud') == 1) & (pl.col('isFraud') == 1)).height / dataset.raw_data.filter(pl.col('isFraud') == 1).height)

# Plot delle categorie
plot_categories(dataset.raw_data)


----HEAD----
shape: (5, 11)
┌──────┬──────────┬──────────┬─────────────┬───┬────────────┬────────────┬─────────┬───────────────┐
│ step ┆ type     ┆ amount   ┆ nameOrig    ┆ … ┆ oldbalance ┆ newbalance ┆ isFraud ┆ isFlaggedFrau │
│ ---  ┆ ---      ┆ ---      ┆ ---         ┆   ┆ Dest       ┆ Dest       ┆ ---     ┆ d             │
│ i64  ┆ str      ┆ f64      ┆ str         ┆   ┆ ---        ┆ ---        ┆ i64     ┆ ---           │
│      ┆          ┆          ┆             ┆   ┆ f64        ┆ f64        ┆         ┆ i64           │
╞══════╪══════════╪══════════╪═════════════╪═══╪════════════╪════════════╪═════════╪═══════════════╡
│ 1    ┆ PAYMENT  ┆ 9839.64  ┆ C1231006815 ┆ … ┆ 0.0        ┆ 0.0        ┆ 0       ┆ 0             │
│ 1    ┆ PAYMENT  ┆ 1864.28  ┆ C1666544295 ┆ … ┆ 0.0        ┆ 0.0        ┆ 0       ┆ 0             │
│ 1    ┆ TRANSFER ┆ 181.0    ┆ C1305486145 ┆ … ┆ 0.0        ┆ 0.0        ┆ 1       ┆ 0             │
│ 1    ┆ CASH_OUT ┆ 181.0    ┆ C840083671  ┆ … ┆ 21182.0    ┆ 0